In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [9]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [11]:
Y = net(torch.rand(4, 8))
Y.mean()    # 因为存储精度的原因，我们仍然可能看到一个非常小的非零数

tensor(2.7940e-09, grad_fn=<MeanBackward0>)

In [12]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.rand(units, ))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias
        return F.relu(linear)

In [13]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.1133, -0.1498,  1.4774],
        [-0.2606,  0.7157, -0.0065],
        [ 0.2815, -1.3443, -0.4077],
        [-0.7531, -0.0306,  0.6773],
        [-0.1868, -0.8793, -1.6970]], requires_grad=True)

In [14]:
linear(torch.rand(2, 5))

tensor([[0.5669, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.8407]], grad_fn=<ReluBackward0>)

In [16]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[16.4580],
        [10.4051]], grad_fn=<ReluBackward0>)